# Readme
This is a demo notebook that shows how the core schema is loaded and how to add data

If you have not installed `djimaging`, please go to project directory and install it via `pip install --editable .`.

In [ ]:
%load_ext autoreload
%autoreload 2

# Data Base

## Prepare dj config

In [ ]:
import os
import datajoint as dj

In [ ]:
username = !whoami
username = username[0]
username

In [ ]:
home_directory = os.path.expanduser("~")
home_directory

In [ ]:
# Set config file
config_file = f'{home_directory}/datajoint/dj_{username}_conf.json'
assert os.path.isfile(config_file), f'Set the path to your config file: {config_file}'

In [ ]:
# Define a schema name or use the default name for your personal test schema
# It should start with ageuler and have some meaningful name after that
schema_name = f"ageuler_{username}_test"

In [ ]:
# Load configuration for user
dj.config.load(config_file)
dj.config['schema_name'] = schema_name
print("schema_name:", dj.config['schema_name'])
dj.conn()

## Create or load schema

In [ ]:
from djimaging.schemas.core_schema import *

In [ ]:
from djimaging.utils.dj_utils import activate_schema

activate_schema(schema=schema, create_schema=True, create_tables=True)
schema

In [ ]:
# Draw the ERD of your schema, this may take a while
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

dj.ERD(schema)

## Upload user

In [ ]:
userinfo = {
    'experimenter': 'DataJointTestData', # Replace this if you want to use your own data
    'data_dir': '/gpfs01/euler/data/Data/DataJointTestData/', # Replace this if you want to use your own data
    'datatype_loc': 0,
    'animal_loc': 1,
    'region_loc': 2,
    'field_loc': 3,
    'stimulus_loc': 4,
    'condition_loc': 5,
}

assert os.path.isdir(userinfo['data_dir'])

In [ ]:
UserInfo().upload_user(userinfo)
UserInfo()

In [ ]:
# Plot the data files in the selected folder
UserInfo().plot1(key=UserInfo().fetch(as_dict=True)[0], show_pre=False, show_raw=False, show_header=True)

## Populate data

### Experiments

In [ ]:
# Before scan
Experiment()

In [ ]:
Experiment().rescan_filesystem(verboselvl=2)

In [ ]:
# After scan
Experiment()

### Fields

In [ ]:
# Before scan
Field()

In [ ]:
Field().rescan_filesystem(verboselvl=2)

In [ ]:
# After scan
Field()

In [ ]:
# If you call plot1 on a djimaging table, it will plot the given key.
# If you pass key=None or no key it will pick a key at random.
# This is implemented for most core tables and can be useful to get a quick impression of the data.
Field().plot1(key=None)

#### Rois

In [ ]:
Roi().populate()
Roi()

In [ ]:
Roi().plot1()

### Stimuli

#### Add default stimuli

In [ ]:
Stimulus()

In [ ]:
# To compute receptive fields, the noise stimulus trace must be loaded and set!

import h5py
import numpy as np

with h5py.File("/gpfs01/euler/data/Resources/Stimulus/noise.h5", "r") as f:
    noise_stimulus = f['stimulusarray'][:].T.astype(int)

In [ ]:
Stimulus().add_nostim(skip_duplicates=True)
Stimulus().add_chirp(spatialextent=1000, stim_name='gChirp', alias="chirp_gchirp_globalchirp", skip_duplicates=True)
Stimulus().add_chirp(spatialextent=300, stim_name='lChirp', alias="lchirp_localchirp", skip_duplicates=True)
Stimulus().add_noise(stim_name='noise', pix_n_x=20, pix_n_y=15, pix_scale_x_um=30, pix_scale_y_um=30, stim_trace=noise_stimulus, skip_duplicates=True)
Stimulus().add_movingbar(skip_duplicates=True)

In [ ]:
Stimulus()

#### Add custom stimuli

In [ ]:
# Replace this with your own stimulus or remove it

def add_cricket(stimulus_table, bkg_color: int, bar_color: int, skip_duplicates=False):

    stim_name = f'cricket{bkg_color}'
    stim_family = 'cricket'
    alias = f"c{bkg_color}_cricket{bkg_color}"

    framerate = 1 / 60.

    stim_dict = {
        "nTrials": 5,
        "tMoveDur_s": 3.0,  # duration of movement (-> traveled distance)
        "barDx_um": 75.0,  # "cricket" dimensions in um (Johnson et al., 2021)
        "barDy_um": 195.0,
        "vel_umSec": 650.0,  # speed of "cricket" in um/sec
        "bkgColor": (bkg_color, bkg_color, bkg_color),
        "barColor": (bar_color, bar_color, bar_color),
        "durFr_s": 1 / 60.0,
        "nFrPerMarker": 3,
    }

    trial_info = np.array([0, 180, 45, 225, 90, 270, 135, 315])

    stimulus_table.add_stimulus(
        stim_name=stim_name, stim_family=stim_family, alias=alias, framerate=framerate,
        isrepeated=True, ntrigger_rep=1,
        stim_dict=stim_dict, trial_info=trial_info,
        skip_duplicates=skip_duplicates, unique_alias=True)

In [ ]:
for bkg_color_ in [63, 127, 255]:
    add_cricket(Stimulus(), bkg_color=bkg_color_, bar_color=0, skip_duplicates=True)

In [ ]:
Stimulus()

#### Presentations

In [ ]:
Presentation().populate()

In [ ]:
Presentation()

In [ ]:
Presentation().plot1(key=None)

### Traces

In [ ]:
Traces().populate()
Traces()

In [ ]:
PreprocessParams().add_default(skip_duplicates=True)
PreprocessParams()

In [ ]:
PreprocessTraces().populate(display_progress=True)
PreprocessTraces()

In [ ]:
PreprocessTraces().plot1()

In [ ]:
Snippets().populate(display_progress=True)
Snippets()

In [ ]:
Snippets().plot1()

In [ ]:
Averages().populate(display_progress=True)
Averages()

In [ ]:
Averages().plot1()

In [ ]:
Averages().plot({"stim_name": "gChirp"})